<a href="https://colab.research.google.com/github/parus-cristatus/tolokapizza/blob/main/tolokaapi/download_all_assignments_from_pool_to_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Docs**: https://toloka.ai/en/docs/api/concepts/result

In [5]:
import requests
import json
import csv
from typing import Generator

In [10]:
endpoint = 'https://sandbox.toloka.dev/api/v1/' # https://toloka.dev/api/v1/ for Production
token = 'AQAxxxxxxxxxxxxxxxxxxxxxxxCxPF6Q' # Toloka OAuth token

In [7]:
# GET call
def get_call(call: str, token: str, endpoint: str, params: dict=None) -> dict:
    """Returns json data from get request.
    
    Arguments:
        call: parameter
        token: toloka token (sandbox or prod)
        endpoint: endpoint (sandbox or prod)
        params: get call params
    
    Examples:
        >>> endpoint = 'https://toloka.dev/api/v1/'
        >>> token = 'KpC_UWW4h17E...'
        >>> get_call('projects', token, endpoint, {'limit': 100})
    """
    headers = {
        "Authorization": "OAuth " + token
    }
    response = requests.get(url=f"{endpoint}{call}", headers=headers, params=params)
    return response.json()

In [9]:
pool = '1181996' # Pool ID where to download accepted assignments from
result = {} 

def params(pool: int, limit: int, status: str='ACCEPTED', id_gt: str=None) ->dict:
    return {'pool_id': pool, 'limit': limit, 'status': status, 'id_gt': id_gt}

def get_assignments(limit: int) ->Generator[dict, None, None]:
    assignments = get_call('assignments', token, endpoint, params(pool, limit))
    last_id = assignments['items'][-1]['id']
    has_more = assignments['has_more']
    while True:
        if has_more:
            yield assignments['items']
            assignments = get_call('assignments', token, endpoint, params(pool, limit, id_gt=last_id))
            last_id = assignments['items'][-1]['id']
            has_more = assignments['has_more']
            
        else:
            yield assignments['items']
            break

all_items = (i for items in get_assignments(300) for i in items) # Max limit for stable work is 300. Default is 50.

result["items"] = [item for item in all_items]
result["length"] = len(result["items"])


with open('accepted_assignments.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(result, indent=4))